In [1]:
pip install warcio

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import warcio
from contextlib import closing
from bs4 import BeautifulSoup
import json

import logging
from IPython.display import HTML
import pandas as pd

In [3]:
! python -m pip install comcrawl

In [4]:
cdx_indexes = requests.get('https://index.commoncrawl.org/collinfo.json').json()

In [5]:
cdx_indexes

[{'id': 'CC-MAIN-2022-05',
  'name': 'January 2022 Index',
  'timegate': 'https://index.commoncrawl.org/CC-MAIN-2022-05/',
  'cdx-api': 'https://index.commoncrawl.org/CC-MAIN-2022-05-index'},
 {'id': 'CC-MAIN-2021-49',
  'name': 'November 2021 Index',
  'timegate': 'https://index.commoncrawl.org/CC-MAIN-2021-49/',
  'cdx-api': 'https://index.commoncrawl.org/CC-MAIN-2021-49-index'},
 {'id': 'CC-MAIN-2021-43',
  'name': 'October 2021 Index',
  'timegate': 'https://index.commoncrawl.org/CC-MAIN-2021-43/',
  'cdx-api': 'https://index.commoncrawl.org/CC-MAIN-2021-43-index'},
 {'id': 'CC-MAIN-2021-39',
  'name': 'September 2021 Index',
  'timegate': 'https://index.commoncrawl.org/CC-MAIN-2021-39/',
  'cdx-api': 'https://index.commoncrawl.org/CC-MAIN-2021-39-index'},
 {'id': 'CC-MAIN-2021-31',
  'name': 'July 2021 Index',
  'timegate': 'https://index.commoncrawl.org/CC-MAIN-2021-31/',
  'cdx-api': 'https://index.commoncrawl.org/CC-MAIN-2021-31-index'},
 {'id': 'CC-MAIN-2021-25',
  'name': 'Ju

In [6]:
pd.options.display.max_colwidth=150
pd.options.display.max_rows=6

In [7]:
df =pd.DataFrame(cdx_indexes)
df

,id,name,timegate,cdx-api
0,CC-MAIN-2022-05,January 2022 Index,https://index.commoncrawl.org/CC-MAIN-2022-05/,https://index.commoncrawl.org/CC-MAIN-2022-05-index
1,CC-MAIN-2021-49,November 2021 Index,https://index.commoncrawl.org/CC-MAIN-2021-49/,https://index.commoncrawl.org/CC-MAIN-2021-49-index
2,CC-MAIN-2021-43,October 2021 Index,https://index.commoncrawl.org/CC-MAIN-2021-43/,https://index.commoncrawl.org/CC-MAIN-2021-43-index
...,...,...,...,...
84,CC-MAIN-2012,Index of 2012 ARC files,https://index.commoncrawl.org/CC-MAIN-2012/,https://index.commoncrawl.org/CC-MAIN-2012-index
85,CC-MAIN-2009-2010,Index of 2009 - 2010 ARC files,https://index.commoncrawl.org/CC-MAIN-2009-2010/,https://index.commoncrawl.org/CC-MAIN-2009-2010-index
86,CC-MAIN-2008-2009,Index of 2008 - 2009 ARC files,https://index.commoncrawl.org/CC-MAIN-2008-2009/,https://index.commoncrawl.org/CC-MAIN-2008-2009-index


In [8]:
df=df[df.id.str.contains('2020')]
df

,id,name,timegate,cdx-api
10,CC-MAIN-2020-50,November 2020 Index,https://index.commoncrawl.org/CC-MAIN-2020-50/,https://index.commoncrawl.org/CC-MAIN-2020-50-index
11,CC-MAIN-2020-45,October 2020 Index,https://index.commoncrawl.org/CC-MAIN-2020-45/,https://index.commoncrawl.org/CC-MAIN-2020-45-index
12,CC-MAIN-2020-40,September 2020 Index,https://index.commoncrawl.org/CC-MAIN-2020-40/,https://index.commoncrawl.org/CC-MAIN-2020-40-index
...,...,...,...,...
16,CC-MAIN-2020-16,March 2020 Index,https://index.commoncrawl.org/CC-MAIN-2020-16/,https://index.commoncrawl.org/CC-MAIN-2020-16-index
17,CC-MAIN-2020-10,February 2020 Index,https://index.commoncrawl.org/CC-MAIN-2020-10/,https://index.commoncrawl.org/CC-MAIN-2020-10-index
18,CC-MAIN-2020-05,January 2020 Index,https://index.commoncrawl.org/CC-MAIN-2020-05/,https://index.commoncrawl.org/CC-MAIN-2020-05-index


In [9]:
!pip install tabulate

In [10]:
print(df.tail(1).to_markdown())

|    | id              | name               | timegate                                       | cdx-api                                             |
|---:|:----------------|:-------------------|:-----------------------------------------------|:----------------------------------------------------|
| 18 | CC-MAIN-2020-05 | January 2020 Index | https://index.commoncrawl.org/CC-MAIN-2020-05/ | https://index.commoncrawl.org/CC-MAIN-2020-05-index |


In [11]:
api_url = df['cdx-api'][11]
api_url

'https://index.commoncrawl.org/CC-MAIN-2020-45-index'

In [12]:
r = requests.get(api_url,
                 params = {
                     'url': '*.covid19.com',
                     'limit': 10,
                     'output': 'json'
                 })
print(r)

<Response [200]>


In [13]:
records = [json.loads(line) for line in r.text.split('\n') if line]

In [14]:
pd.DataFrame(records)

,urlkey,timestamp,url,mime,mime-detected,status,digest,length,offset,filename,languages,encoding
0,"com,covid19)/",20201024225240,https://covid19.com/,text/html,application/xhtml+xml,200,IVF5NCBANCRJ6OYOJJSEPDV3HTBW6B2G,2091,279345234,crawl-data/CC-MAIN-2020-45/segments/1603107885059.50/warc/CC-MAIN-20201024223210-20201025013210-00127.warc.gz,eng,UTF-8
1,"com,covid19)/robots.txt",20201024225240,https://covid19.com/robots.txt,text/html,text/html,404,2ZF25EIJD3NGU5JS5PWANKTQRE5XTYPY,680,1526905,crawl-data/CC-MAIN-2020-45/segments/1603107885059.50/robotstxt/CC-MAIN-20201024223210-20201025013210-00554.warc.gz,NaN,NaN


In [15]:
print(pd.DataFrame(records).head().to_markdown())

|    | urlkey                  |      timestamp | url                            | mime      | mime-detected         |   status | digest                           |   length |    offset | filename                                                                                                           | languages   | encoding   |
|---:|:------------------------|---------------:|:-------------------------------|:----------|:----------------------|---------:|:---------------------------------|---------:|----------:|:-------------------------------------------------------------------------------------------------------------------|:------------|:-----------|
|  0 | com,covid19)/           | 20201024225240 | https://covid19.com/           | text/html | application/xhtml+xml |      200 | IVF5NCBANCRJ6OYOJJSEPDV3HTBW6B2G |     2091 | 279345234 | crawl-data/CC-MAIN-2020-45/segments/1603107885059.50/warc/CC-MAIN-20201024223210-20201025013210-00127.warc.gz      | eng         | UTF-8      |
|  1